### Connecting to Cloud Object Storage (S3)

In [ ]:
# https://pypi.python.org/pypi/ibmos2spark
#!pip install ibmos2spark
# https://pypi.python.org/pypi/ibm-cos-sdk
#!pip install ibm-cos-sdk

In [ ]:
# source code:   https://github.com/ibm-watson-data-lab/ibmos2spark
# documentation: https://github.com/ibm-watson-data-lab/ibmos2spark/tree/master/python#usage
import ibmos2spark

# documentation: https://console.bluemix.net/docs/services/cloud-object-storage/libraries/python.html#python
import boto3
from botocore.client import Config

In [ ]:
# TODO:
# (1) add a Cloudant connection to this project
# (2) use "insert to code" to insert Cloudant connection credentials into this cell
# (3) rename the "credentials_X" variable to "credentials"


### Explore the Cloud Object Storage instance

In [ ]:
# connect to Cloud Object Storage
cos = boto3.client('s3',
                    ibm_api_key_id=credentials['api_key'],
                    ibm_service_instance_id=credentials['resource_instance_id'],
                    ibm_auth_endpoint=credentials['iam_url'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials['url'])

In [ ]:
# query Cloud Object Storage
bucket_name = None
object_key = None

try:
    # retrieve bucket list
    buckets = cos.list_buckets()['Buckets']
    print 'Number of buckets in this Cloud Object Storage instance: {}'.format(len(buckets))
    
    # find the first bucket that contains an object that's less than 1MB in size
    for bucket in buckets:
        try:
            list_objects_response = cos.list_objects(Bucket=bucket['Name'])
            if list_objects_response.get('Contents'):
                for object in list_objects_response['Contents']:
                    if object['Key'].endswith('.csv') and object['Size'] < 1024 * 1024:
                        bucket_name = bucket['Name']
                        object_key = object['Key']
                        break
        except Exception as nsb:                 
           pass
        finally:
            if bucket_name is not None and object_key is not None:
               print 'Bucket {} contains a small csv object named {}'.format(bucket_name, object_key)
               break
    
except Exception as e:
    print "Query: ERROR - {}".format(e)  
        


### Download an object from a bucket

In [ ]:
# try to connect
try:    
    creds = {
        'endpoint': credentials['url'],
        'api_key': credentials['api_key'],
        'service_id': credentials['resource_instance_id']
    }

    configuration_name = 'os_bluemix_cos_config'
    
    dcos = ibmos2spark.CloudObjectStorage(sc, 
                                          creds,
                                          configuration_name=configuration_name,
                                          cos_type='bluemix_cos')
    print "Connection: OK"
    
    if bucket_name is not None and object_key is not None:
        data_url = dcos.url(object_key, bucket_name)
        data = sc.textFile(data_url)
        print 'The file contains {} rows'.format(data.count())
    
except Exception as e:
    print "Connection: ERROR - {}".format(e)